In [1]:
pip install torch>=2.0.0

In [2]:
pip install torchvision>=0.15.0

In [3]:
pip install tqdm>4.65.0

In [4]:
pip install torchsummary>=1.5.1

In [8]:
# Import required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
from torchsummary import summary

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Initial layers
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)

        # First block
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)

        # Second block
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.conv5 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(32)

        # Output layer
        self.conv6 = nn.Conv2d(32, 10, 1)

    def forward(self, x):
        # Initial block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, 0.1)

        # First block
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, 0.1)

        # Second block
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))

        # Output block
        x = self.conv6(x)
        x = F.avg_pool2d(x, 7)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [10]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item():.4f} batch_id={batch_idx}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    return accuracy

In [11]:
# Set random seed for reproducibility
torch.manual_seed(1)

# Check if CUDA is available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Training settings
batch_size = 128
epochs = 15
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

print("Downloading and loading MNIST dataset...")
# Data augmentation for training
train_transforms = transforms.Compose([
    transforms.RandomRotation((-7.0, 7.0)),  # Random rotation
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random shift
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Test transforms (no augmentation)
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                transform=train_transforms),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                transform=test_transforms),
    batch_size=batch_size, shuffle=True, **kwargs)

print("Initializing model...")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# Initialize optimizer with cosine annealing scheduler
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# Training loop
best_accuracy = 0.0
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}")
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)
    scheduler.step()

    # Save best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'New best accuracy: {best_accuracy:.2f}%')

print(f'\nBest Test accuracy: {best_accuracy:.2f}%')

Initializing model...
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 14, 14]           2,320
       BatchNorm2d-4           [-1, 16, 14, 14]              32
            Conv2d-5           [-1, 16, 14, 14]           2,320
       BatchNorm2d-6           [-1, 16, 14, 14]              32
            Conv2d-7             [-1, 32, 7, 7]           4,640
       BatchNorm2d-8             [-1, 32, 7, 7]              64
            Conv2d-9             [-1, 32, 7, 7]           9,248
      BatchNorm2d-10             [-1, 32, 7, 7]              64
           Conv2d-11             [-1, 10, 7, 7]             330
Total params: 19,242
Trainable params: 19,242
Non-trainable params: 0
----------------------------------------------------------------
Input size

loss=0.0248 batch_id=468: 100%|██████████| 469/469 [00:31<00:00, 14.75it/s]



Test set: Average loss: 0.0990, Accuracy: 9692/10000 (96.92%)

New best accuracy: 96.92%

Epoch 2


loss=0.0971 batch_id=468: 100%|██████████| 469/469 [00:32<00:00, 14.31it/s]



Test set: Average loss: 0.0844, Accuracy: 9714/10000 (97.14%)

New best accuracy: 97.14%

Epoch 3


...


Test set: Average loss: 0.0239, Accuracy: 9922/10000 (99.22%)

New best accuracy: 99.22%

Epoch 14


loss=0.0085 batch_id=468: 100%|██████████| 469/469 [00:31<00:00, 14.92it/s]



Test set: Average loss: 0.0220, Accuracy: 9937/10000 (99.37%)

New best accuracy: 99.37%

Epoch 15


loss=0.0232 batch_id=468: 100%|██████████| 469/469 [00:31<00:00, 15.03it/s]



Test set: Average loss: 0.0210, Accuracy: 9932/10000 (99.32%)


Best Test accuracy: 99.37%
